This Notebook 

1. downloads the EEW earthquake catalog from ONC
2. converts catalog into GeoJson that can be viewed on Github
3. downloads the JSON definition of each event in the catalog into a file under the **events** folder

The callouts on the [GeoJson map](https://github.com/MHee/virtual_earthquake_catalog_test/blob/main/earthquake_catalog.geojson) include the URLs that
1. Should render the json file on Github using the ONC interface
2. The plain URL of the json file on Github
3. The link to the original event detail page using the ONC DB 

In [1]:
import requests

In [2]:
from getpass import getpass

In [3]:
# Use token from https://data.oceannetworks.ca/Profile
token = getpass('Your ONC API Token:')

Your ONC API Token: ····································


## GeoJSON
Classes that help to generate GeoJson without dependencies

In [5]:
# https://geojson.org/
import json

class GeoJsonObj(object):
    def __init__(self, obj_type=None):
        self.json_obj={'type': obj_type}
        
    def __str__(self):
        return str(self.json_obj)
    
    def __repr__(self):
        return self.__str__()
    
    def to_json(self):
        return json.dumps(self.json_obj, sort_keys=False, indent=2)
    
class Point(GeoJsonObj):
    def __init__(self, longitude, latitude):
            GeoJsonObj.__init__(self, obj_type='Point')
            self.json_obj['coordinates'] = [longitude, latitude]
            
class Feature(GeoJsonObj):
    def __init__(self, geometry=None, properties=None):
            GeoJsonObj.__init__(self, obj_type='Feature')
            self.json_obj['geometry'] = geometry.json_obj
            self.json_obj['properties'] = properties
            
class FeatureCollection(GeoJsonObj):
    def __init__(self, features=[]):
            GeoJsonObj.__init__(self, obj_type='FeatureCollection')
            self.features = list()
            self.json_obj['features'] = self.features
            for feature in features:
                self.append(feature)
            
    def append(self, feature):
        self.features.append(feature.json_obj)
            
    

## ONC Earthquake Catalog to GeoJSON

In [23]:
r = requests.get('https://data.oceannetworks.ca/EarthquakeCatalogService', params={'operation': 1, 'token': token})
r

<Response [200]>

In [24]:
earthquakes = r.json()['payload']

In [25]:
earthquake_collection = FeatureCollection()

for earthquake in earthquakes:
    p = Point(earthquake['longitude'], earthquake['latitude'])
    
    gh_url = f'https://raw.githubusercontent.com/MHee/virtual_earthquake_catalog_test/main/events/eq_{earthquake["earthquakeId"]}.json'
    url = f'https://data.oceannetworks.ca/EarthquakeDetail?earthquakeURL={requests.utils.quote(gh_url)}'
    url_orig = f'https://data.oceannetworks.ca/EarthquakeDetail?earthquakeId={earthquake["earthquakeId"]}'
    
    earthquake['url'] = url
    earthquake['gh_url'] = gh_url
    earthquake['url_orig'] = url_orig
    
    earthquake_collection.append(Feature(p, earthquake))


with open('earthquake_catalog.geojson', 'w') as catalog_file:    
    catalog_file.write(earthquake_collection.to_json())

## Dump all earthquakes in catalog to JSON

In [20]:
params={'operation': 1, 'earthquakeId': None, 'token': token}

for earthquake in earthquakes:
    out_file = f'./events/eq_{earthquake["earthquakeId"]}.json'
    params['earthquakeId'] = earthquake['earthquakeId']
    print(out_file)
    r = requests.get('https://data.oceannetworks.ca/EarthquakeDetailService', params=params)
    if r.ok:
        with open(out_file, 'w') as json_file:
            json.dump(r.json()['payload'], json_file, indent=2)
    else:
        print(f'Something when wrong with {r.url} !!!')

./events/eq_1.json
./events/eq_2.json
./events/eq_3.json
./events/eq_4.json
./events/eq_5.json
./events/eq_6.json
./events/eq_7.json
./events/eq_8.json
./events/eq_9.json
./events/eq_10.json
./events/eq_11.json
./events/eq_12.json
./events/eq_13.json
./events/eq_14.json
./events/eq_15.json
./events/eq_16.json
./events/eq_17.json
./events/eq_18.json
